In [ ]:
!pip install -r requirements.txt
# !pip install git+https://github.com/PyTorchLightning/pytorch-lightning
!pip install kobart-transformers
!pip install nlp

     |████████████████████████████████| 1.9 MB 26.7 MB/s 
     |████████████████████████████████| 7.4 MB 31.9 MB/s 
     |████████████████████████████████| 813 kB 55.7 MB/s 
     |████████████████████████████████| 895 kB 41.2 MB/s 
     |████████████████████████████████| 3.3 MB 57.4 MB/s 
     |████████████████████████████████| 75 kB 4.0 MB/s 
     |████████████████████████████████| 4.2 MB 60.6 MB/s 
     |████████████████████████████████| 177 kB 63.8 MB/s 
     |████████████████████████████████| 111 kB 60.7 MB/s 
     |████████████████████████████████| 122 kB 59.1 MB/s 
     |████████████████████████████████| 786 kB 62.7 MB/s 
     |████████████████████████████████| 368 kB 63.4 MB/s 
     |████████████████████████████████| 118 kB 64.1 MB/s 
     |████████████████████████████████| 636 kB 62.7 MB/s 
     |████████████████████████████████| 829 kB 53.9 MB/s 
     |████████████████████████████████| 234 kB 55.8 MB/s 
     |████████████████████████████████| 10.6 MB 46.2 MB/s 
     |█████████

     |████████████████████████████████| 1.7 MB 36.6 MB/s 
     |████████████████████████████████| 243 kB 49.9 MB/s 


In [ ]:
# imports
import transformers
from torch.utils.data import DataLoader, TensorDataset, random_split, RandomSampler, Dataset
import pandas as pd
import numpy as np

import torch.nn.functional as F
import pytorch_lightning as pl
import torch
from pytorch_lightning.callbacks import ModelCheckpoint

import math
import random
import re
import argparse

In [ ]:
from transformers import (
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    Trainer,
    TrainingArguments,
)

2021-07-23 06:54:10.544128: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


# 모델, 토크나이저 정의

In [ ]:
from kobart_transformers import get_kobart_model, get_kobart_tokenizer
from transformers import PreTrainedTokenizerFast
from transformers import BartForSequenceClassification
# tokenizer = get_kobart_tokenizer()
# model = get_kobart_model()
tokenizer = AutoTokenizer.from_pretrained("hyunwoongko/kobart")
model = AutoModelForSeq2SeqLM.from_pretrained("hyunwoongko/kobart")

In [ ]:
tokenizer(["한국어", "모델을", "소개합니다."], truncation=True, padding=True)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


{'input_ids': [[15274, 11763, 3], [18181, 12007, 3], [11319, 9006, 20357]], 'attention_mask': [[1, 1, 0], [1, 1, 0], [1, 1, 1]]}

# 데이터 로드 및 병합

In [ ]:
import nlp
data = pd.read_excel("bonobono.xlsx")
dataset = data.iloc[:,2:4]
dataset = dataset.dropna(axis=0,how='any')

In [ ]:
data1 = pd.read_excel("sea1.xlsx")
dataset1 = data1.iloc[:,2:4]
dataset1 = dataset1.dropna(axis=0,how='any')

In [ ]:
data2 = pd.read_excel("sea2.xlsx")
dataset2 = data2.iloc[:,2:4]
dataset2 = dataset2.dropna(axis=0,how='any')

In [ ]:
# data3 = pd.read_excel("chatbot.xlsx")
# dataset3 = data3.iloc[:,1:3]
# dataset3 = dataset3.dropna(axis=0,how='any')

# 데이터 이상치가 많음

In [ ]:
dataset = dataset.append((dataset1,dataset2))

In [ ]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1405 entries, 0 to 443
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   반말A     1405 non-null   object
 1   존댓말A    1405 non-null   object
dtypes: object(2)
memory usage: 32.9+ KB


In [ ]:
train_dataset = nlp.Dataset.from_pandas(dataset[:1100])
val_dataset = nlp.Dataset.from_pandas(dataset[1100:])

In [ ]:
train_dataset

Dataset(features: {'반말A': Value(dtype='string', id=None), '존댓말A': Value(dtype='string', id=None), '__index_level_0__': Value(dtype='int64', id=None)}, num_rows: 1100)

In [ ]:
def map_to_encoder_decoder_inputs(batch):
    inputs = tokenizer(batch["반말A"], padding="max_length", truncation=True, max_length=40) # 반말
    outputs = tokenizer(batch["존댓말A"], padding="max_length", truncation=True, max_length=40) # 존댓말

    batch["input_ids"] = inputs.input_ids
    batch["attention_mask"] = inputs.attention_mask

    # batch["decoder_input_ids"] = outputs.input_ids
    batch["labels"] = outputs.input_ids
    # batch["labels"] = [
    #     [-100 if token == tokenizer.pad_token_id else token for token in labels] for labels in batch["labels"]
    # ]
    # model.generate(inputs.input_ids)
    # batch["decoder_attention_mask"] = outputs.attention_mask
    
    # print("inputs:",inputs)
    # print("outputs:",outputs)
    assert all([len(x) == 40 for x in inputs.input_ids])
    assert all([len(x) == 40 for x in outputs.input_ids])

    return batch

def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

    rouge_output = rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge2"])["rouge2"].mid

    return {
      "rouge2_precision": round(rouge_output.precision, 4),
      "rouge2_recall": round(rouge_output.recall, 4),
      "rouge2_fmeasure": round(rouge_output.fmeasure, 4),
    }

In [ ]:
batch_size = 16

train_dataset = train_dataset.map(
    map_to_encoder_decoder_inputs, batched=True, batch_size=batch_size, remove_columns=["반말A", "존댓말A"],
)
train_dataset.set_format(
    type="torch", columns=["input_ids", "attention_mask","labels"],
)

val_dataset = val_dataset.map(
    map_to_encoder_decoder_inputs, batched=True, batch_size=batch_size, remove_columns=["반말A", "존댓말A"],
)
val_dataset.set_format(
    type="torch", columns=["input_ids", "attention_mask","labels"],
)

In [ ]:
model.to('cuda')

BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(30000, 768, padding_idx=3)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(30000, 768, padding_idx=3)
      (embed_positions): BartLearnedPositionalEmbedding(1028, 768, padding_idx=3)
      (layers): ModuleList(
        (0): BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
   

In [ ]:
# !nvidia-smi

In [ ]:
train_dataset

Dataset(features: {'__index_level_0__': Value(dtype='int64', id=None), 'input_ids': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None), 'attention_mask': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None), 'labels': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None)}, num_rows: 1100)

In [ ]:
base_dir = 'content/'
# checkpoint = ModelCheckpoint(filepath=base_dir + 'checkpoint_files_1/')

training_args = TrainingArguments(
    output_dir="./",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    # evaluate_during_training=True,
    do_train=True,
    # do_eval=True,
    logging_steps=5,
#     save_steps=5,
    eval_steps=20,
    overwrite_output_dir=True,
    warmup_steps=100,
    save_total_limit=1,
    num_train_epochs=5,
    # checkpoint_callback = checkpoint,
)

trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)



In [ ]:
trainer.train()

Step,Training Loss
5,17.305600
10,15.046900
15,13.323900
20,10.517600
25,9.116300
30,7.501200
35,5.836200
40,4.461800
45,3.911000
50,3.331900


TrainOutput(global_step=345, training_loss=1.6621123881443687, metrics={'train_runtime': 75.1108, 'train_samples_per_second': 4.593, 'total_flos': 163495157760000, 'epoch': 5.0})

In [ ]:
inputs = tokenizer.encode("오늘 날씨 짱 좋아!!!", return_tensors='pt')
model.to('cpu')
pred = model.generate(inputs)
tokenizer.decode(pred[0])

'<usr>오늘 날씨 날씨 짱 좋아요<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>'

In [ ]:
torch.save(model.state_dict(), 'kobart')

In [ ]:
model.load_state_dict(torch.load('kobart'))
model.eval()
type(model)

transformers.models.bart.modeling_bart.BartForConditionalGeneration

In [ ]:
class Styleformer():

    def __init__(self,  style=0):
        from transformers import AutoTokenizer
        from transformers import AutoModelForSeq2SeqLM
#         from styleformer import Adequacy

        self.style = style
        self.adequacy = Adequacy()
        self.model_loaded = False

        if self.style == 0:
            self.ctf_tokenizer = AutoTokenizer.from_pretrained("hyunwoongko/kobart")
            model.load_state_dict(torch.load('content'))
            self.ctf_model = model
#             print("test->",self.ctf_tokenizer("today is a sunny day"))
            
            print("Casual to Formal model loaded...")
            self.model_loaded = True
        else:
            print("Only CTF, FTC, ATP and PTA are supported in the pre-release...stay tuned")

    def transfer(self, input_sentence, inference_on=0, quality_filter=0.95, max_candidates=5):
        if self.model_loaded:
            if inference_on == 0:
                device = "cpu"
            elif inference_on == 1:
                device = "cuda:0"  
            else:  
                device = "cpu"
                print("Onnx + Quantisation is not supported in the pre-release...stay tuned.")

            if self.style == 0:
                output_sentence = self._casual_to_formal(input_sentence, device, quality_filter, max_candidates)
                return output_sentence     
        else:
            print("Models aren't loaded for this style, please use the right style during init")  


    def _casual_to_formal(self, input_sentence, device, quality_filter, max_candidates):
        ctf_prefix = "transfer Casual to Formal: "
        src_sentence = input_sentence
        input_sentence = ctf_prefix + input_sentence
        input_ids = self.ctf_tokenizer.encode(input_sentence, return_tensors='pt')
        self.ctf_model = self.ctf_model.to(device)
        input_ids = input_ids.to(device)

        
        preds = self.ctf_model.generate(
            input_ids,
            do_sample=True, 
            max_length=32, 
            top_k=50, 
            top_p=0.95, 
            early_stopping=True,
            num_return_sequences=max_candidates)
        print("preds : ",preds)
        gen_sentences = set()
        for pred in preds:
            gen_sentences.add(self.ctf_tokenizer.decode(pred, skip_special_tokens=True).strip())
        print("gensentences",gen_sentences)
        adequacy_scored_phrases = self.adequacy.score(src_sentence, list(gen_sentences), quality_filter, device)
        ranked_sentences = sorted(adequacy_scored_phrases.items(), key = lambda x:x[1], reverse=True)
        print(adequacy_scored_phrases)
        print("ranked",ranked_sentences)
        if len(ranked_sentences) > 0:
            return ranked_sentences[0][0]
        else:
            return None


     
        return self.pta_tokenizer.decode(preds[0], skip_special_tokens=True).strip()      

    


In [ ]:
class Adequacy():

  def __init__(self, model_tag='prithivida/parrot_adequacy_on_BART'):
    from transformers import AutoModelForSequenceClassification, AutoTokenizer
    from transformers import PreTrainedTokenizerFast
    # self.nli_model = BartModel.from_pretrained(model_tag)
    # self.tokenizer = PreTrainedTokenizerFast.from_pretrained(model_tag)
    
    self.nli_model = AutoModelForSequenceClassification.from_pretrained(model_tag)
    self.tokenizer = AutoTokenizer.from_pretrained(model_tag)
    
  def filter(self, input_phrase, para_phrases, adequacy_threshold, device="cpu"):
      top_adequacy_phrases = []
      for para_phrase in para_phrases:
        x = self.tokenizer.encode(input_phrase, para_phrase, return_tensors='pt',truncation_strategy='only_first')
        self.nli_model = self.nli_model.to(device)
        logits = self.nli_model(x.to(device))[0]
        # we throw away "neutral" (dim 1) and take the probability of "entailment" (2) as the adequacy score
        entail_contradiction_logits = logits[:,[0,2]]
        probs = entail_contradiction_logits.softmax(dim=1)
        prob_label_is_true = probs[:,1]
        adequacy_score = prob_label_is_true[0].item()
        if adequacy_score >= adequacy_threshold:
            top_adequacy_phrases.append(para_phrase)
      return top_adequacy_phrases

  def score(self, input_phrase, para_phrases, adequacy_threshold, device="cpu"):
      adequacy_scores = {}
      for para_phrase in para_phrases:
        x = self.tokenizer.encode(input_phrase, para_phrase, return_tensors='pt',truncation_strategy='only_first')
        self.nli_model = self.nli_model.to(device)
        logits = self.nli_model(x.to(device))[0]
        # we throw away "neutral" (dim 1) and take the probability of "entailment" (2) as the adequacy score
        entail_contradiction_logits = logits[:,[0,2]]
        probs = entail_contradiction_logits.softmax(dim=1)
#         print("logits-----",logits)
#         print("entail",entail_contradiction_logits)
#         print("probs",probs)
        prob_label_is_true = probs[:,1]
        print("prob_label_is_true : ",prob_label_is_true )
        adequacy_score = prob_label_is_true[0].item()
        if adequacy_score >= adequacy_threshold:
          adequacy_scores[para_phrase] = adequacy_score
#         print(adequacy_scores)
      return adequacy_scores      

In [ ]:
# from styleformer import Styleformer
import torch
import warnings
warnings.filterwarnings("ignore")

'''
#uncomment for re-producability
def set_seed(seed):
  torch.manual_seed(seed)
  if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

set_seed(1234)
'''

# style = [0=Casual to Formal, 1=Formal to Casual, 2=Active to Passive, 3=Passive to Active etc..]
sf = Styleformer(style = 0)

source_sentences = [
"안녕, 나는 파새라고 해",
"그야 있었겠지, 너도 동의하지?",
"응, 맞아. 다들 참기만 해.",

]

for source_sentence in source_sentences:
    target_sentence = sf.transfer(source_sentence)
    print("-" *100)
    print("[Casual] ", source_sentence)
    print("-" *100)
    if target_sentence is not None:
        print("[Formal] ",target_sentence)
        print()
    else:
        print("No good quality transfers available !")


Some weights of the model checkpoint at prithivida/parrot_adequacy_on_BART were not used when initializing BartForSequenceClassification: ['model.encoder.version', 'model.decoder.version']
- This IS expected if you are initializing BartForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BartForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Casual to Formal model loaded...
preds :  tensor([[    2,   315,   313, 15195,   314,   301, 14879, 14664, 16884, 19512,
         21235, 15418, 21738, 17510, 21579, 26545, 15585,   257,     3,     3,
             3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
             3,     3],
        [    2,   315,   313, 15195,   314,   301, 14879, 14664, 19783, 23795,
           315, 21579, 15418, 15463,   308, 15585,   257, 22465,   243, 16530,
         14240, 11229, 14218, 14088, 14543,     3,     3,     3,     3,     3,
             3,     3],
        [    2,   315,   313, 15195,   314,   301, 14879, 14664, 19542,   316,
         15585, 27141, 15418, 15463,   308, 15585,   257, 22465,   243, 16530,
         14240, 11229, 14218, 14088, 14543,     3,     3,     3,     3,     3,
             3,     3],
        [    2,   315,   313, 15195,   314,   301, 14879, 14664, 19783,   313,
             3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
             3,  